# **Off-Peak Friday Fares Pilot Review**

##### **Draft - Confidential

## **Introduction:**

In response to the observed drops in buying and selling on Fridays compared to other days of the week (coined the "Friday Effect"), the Mayor of London introduced a pilot of the Off-Peak Fare Fridays promo which ran from March 8 (Week 10) to May 31 (Week 22), 2024. The promo aimed to encourage more people back into the city for a much needed boost in busyness to accelerate Londer's wider economic growth.

## **Background:**

A prior study on the Friday Effect at daytime in the Central Activity Zone (CAZ) had revealed that pre-COVID (Feb 2020) Friday activity was lower than the Wednesday, but had since dropped further in November 2023. It was however observed that Friday footfall was in fact increasing, albeit at a slower rate than the midweek.

Daytime spend in the CAZ on the other hand was lower on Fridays than on Wednesdays in November 2023. However, Friday daytime spend in the CAZ had increased at a marginally higher year-on-year rate compared to Wednesdays.   

These observations point to the fact that in the CAZ, although Fridays have seen less growth compared to Wednesdays in terms of footfall, Friday spend has seemingly grown on par with Wednesday spend. This variation between footfall growth and spend growth could indicate that footfall need only increase marginally for a significant increase in spend to occur.

## **Scope:**

In this study, we use the Mastercard spend index to assess changes in spend patterns during the pilot period of the reduced fares promo and thus potentially guage how much of an impact (if any) the promo had across commercial hotspots. Our focus on the CAZ and town centre BIDs takes into account the noteworthy concentration of commercial activity in these areas relative to other industrial or residential areas. The study also focuses on daytime (6am to 6pm) spend patterns in the geographical locations of interest as we had seen stronger nighttime spend activity in the prior study and thus, less of a Friday Effect.

To accurately measure growth during the pilot period, we have compare spend from week 10 to week 22 in 2024 with the same weeks in 2023 to measure year-on-year change patterns.

## **Data:**

The spend data used for this analysis was supplied in aggregated and anonymized format by Mastercard and adjusted for inflation and cash to card shift by the Greater London Authority (GLA).

In [ ]:
# import libraries, define functions, read in files
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
import geopandas as gpd
from itertools import chain
from IPython.display import Markdown, display
import warnings
warnings.filterwarnings('ignore')

# read in lookup, cpi & spend files, merge spend files and adjust for spend and cash-card shift
cpi_table = pd.read_csv("...")
quad_caz_lookup = pd.read_excel("...")
spend_2022 = pd.read_csv("...")
spend_2023 = pd.read_csv("...")
spend_2024 = pd.read_csv("...")
bids = pd.read_excel('...')

# read in shape files for spatial plot
lbs = gpd.read_file("...")
cazshp = gpd.read_file("...")
bt_gdf = gpd.read_file("...")
bid_shp = gpd.read_file("...")

# define company styling colours
colours = ['rgb(xx, xx, xx)', 'rgb(xx, xx, xx)', 'rgb(xx, xx, xx)',
           'rgb(xx, xx, xx)', 'rgb(xx, xx, xx)', 'rgb(xx, xx, xx)',
           'rgb(xx, xx, xx)', 'rgb(xx, xx, xx)', 'rgb(xx, xx, xx)',
           'rgb(xx, xx, xx)']

# list bids for appendix analysis
hsds_bids = ['BID 1', 'BID 2', 'BID 3',  'BID 4',  'BID 5',  'BID 6',  'BID 7',
             'BID 8', 'BID 9', 'BID 10', 'BID 11', 'BID 12', 'BID 13', 'BID 14',
             'BID 15','BID 16']

def printmd(string):
  """convert string to markdown"""
    display(Markdown(string))

class C:
  """colours for text highlighting"""
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   b = '\033[1m'
   UNDERLINE = '\033[4m'
   e = '\033[0m'

time_of_day = 'daytime'
# set selection parameters
if time_of_day == 'midday':
    hours = ["'12-15"]
elif time_of_day == 'daytime':
    hours = ["'06-09", "'09-12", "'12-15", "'15-18"]
elif time_of_day == 'nighttime':
    hours = ["'18-21", "'21-24", "'00-03", "'03-06"]
elif time_of_day == 'tfl peak hours':
    hours = ["'06-09", "'09-12", "'15-18", "'18-21"]
elif time_of_day == 'full-day':
    hours = ["'00-03", "'03-06", "'06-09", "'09-12", "'12-15", "'15-18", "'18-21", "'21-24"]
else:
    hours = time_of_day

weekdays_fri = ['rgb(215,216,217)', 'rgb(174,177,180)', 'rgb(134,139,142)', 'rgb(81,90,94)',colours[1]]
weekdays = ['rgb(215, 216, 217)', 'rgb(174,177,180)', colours[0], 'rgb(134,139,142)', colours[1]]
renderer = 'jpeg' # set plotly chart renderer format

# set watermark for confidential charts
pio.templates["draft"] = go.layout.Template(
    layout_annotations = [
        dict(
            name = "draft watermark",
            text = "DRAFT - CONFIDENTIAL",
            textangle = -30,
            opacity = 0.1,
            font = dict(color="black", size=100),
            xref = "paper",
            yref = "paper",
            x = 0.5,
            y = 0.5,
            showarrow = False,
        )
    ]
)

# function for inflation adjustment
def inflation_adjust(spend, cpi_table, reindexing_year=None, cpi_col_name='cpi_index'):
    '''
    Adjusts spend columns 'txn_amt' and 'avg_spend_amt by monthly ONS inflation rates.
    spend: spend table of MC 3-hourly data
    cpi_table: imported and cleaned CPIH table from ONS
    reindexing year (optional): the year that you want to use as cpi_index = 100. If none,
    does not reindex beyond ONS's existing 2015=100 reindex
    '''

    # Reindex to a chosen baseline year, otherwise skip
    if reindexing_year is not None:
        reindex = cpi_table[cpi_table['yr']==reindexing_year][cpi_col_name].mean()
        cpi_table[cpi_col_name] = cpi_table[cpi_col_name]/reindex * 100
    else:
        pass

    # Join spend data with cpi data
    spend = pd.merge(spend, cpi_table, how = 'left', on = ['yr','month'])

    # If spend data is more recent than cpi data, there will be NaNs. Fill them with the latest
    # available cpi index
    max_year = cpi_table['yr'].max()
    max_month = cpi_table[(cpi_table['yr'] == max_year)]['month'].max()
    spend[cpi_col_name].fillna(cpi_table[(cpi_table['month']==max_month) &
                                         (cpi_table['yr']==max_year)][cpi_col_name])

    # Adjust
    spend['txn_amt'] = spend['txn_amt']/spend[cpi_col_name] * 100
    #spend['avg_spend_amt'] = spend['avg_spend_amt']/spend['cpi_index'] * 100

    return spend

# Adjust for cash to card shift
def mcard_adjust(spend, adj_col = 'adjustment_factor_retail'):
    '''
    Adjusts spend column 'txn_amt' by monthly correction factor generated from Spending Pulse data.

    Parameters
    ----------
    spend: spend table of MC 3-hourly/weekly data at quad level (data needs a 'yr' and 'month' column)
    Needs to be adjusted for inflation separately

    Returns
    --------
    Spend data with adjusted txn_amt column
    '''
    adjustment_factor = pd.read_csv(".../mcard_adjustment_factor.csv")
    inner_outer_quad = pd.read_csv(".../Inner_outer_quad_lookup.csv")

    # Add inner_outer to spend data
    spend = pd.merge(spend,inner_outer_quad,how='left',on='quad_id')

    # Join spend data with mcard adjustment data
    spend = pd.merge(spend,adjustment_factor,how='left',on=['yr','month','inner_outer']) # need to merge on inner vs outer too

    # If spend data is more recent than Spending Pulse, there will be NaNs. Fill them with the latest available mcard_adjustment
    spend = spend.sort_values(by=['inner_outer','count_date'])
    spend[adj_col] = spend[adj_col].fillna(method='ffill')

    # Adjust
    spend['txn_amt'] = spend['txn_amt'] / spend[adj_col]

    spend.drop(columns=['inner_outer'],inplace=True) # remove inner_outer column

    return spend

def spend_indexer(df: pd.DataFrame):
  """function to index friday spend to wednesdays & plot March weekday spend"""
    for i, cat in enumerate(['Spend Amount', 'Spend Volume']):
        basis = df[df['day'] == 'Wednesday'][cat].values[0]
        df[cat + ' idx'] = df.apply(lambda x: x[3+i]/basis * 100, axis = 1)
    df.sort_values(by = 'daynumber', ascending = True, inplace = True)
    return df

def bid_spatial_plot(data, vis_col, title, cmap = 'RdYlGn', figsize = (12, 6), vmin = None, vmax = None, saveas = None):
  """function to plot distribution of any variable across the bids"""
    map_df = pd.merge(bid_shp, data, how = 'left', on = "bid_name")
    # map to depict midday resident footfall proportions at a given min and max range
    ax = map_df.plot(column = vis_col, figsize = figsize, alpha = 1, legend = True, cmap = cmap, vmin = vmin, vmax = vmax,
                     legend_kwds = { "orientation": "vertical"}, missing_kwds = {"color": "lightgrey", "label": "Non HSDS BID"})
    #turn off axis. Not relevant
    ax.axis('off')
    #plot borough boundaries
    lbs.plot(ax = ax, alpha = 0.4, facecolor = "none", edgecolor = "#cacaca")
    #plot caz boundary
    cazshp.plot(ax = ax, facecolor = "none", edgecolor = "#6da7de", linewidth = 1.2)
    #set plot title
    ax.set_title(title, fontweight = "bold", fontsize = 15)
    ax.annotate(text = '**Spend Index Aggregated & Anonymised by Mastercard \n   Inflation & Cash-Card Shift Adjusted by the GLA     ',
                xy = [532772.752, 154000],
                ha = 'left',
                size = 8)
    ax.annotate('DRAFT - CONFIDENTIAL', xy = [520000, 180000], fontsize = 20, color = (0, 0, 0, 0.1))
    #save figure if needed
    if saveas != None:
        plt.savefig(saveas + '.png', format = 'png')
    plt.show()

In [ ]:
# preprocess spend data
spend = pd.concat([spend_2022, spend_2023, spend_2024])
spend = spend.merge(quad_caz_lookup, on = 'quad_id', how = 'left')

# extract timeseries features and adjust for inflation
spend['count_date']  = pd.to_datetime(spend['count_date'])
spend['daynumber']   = spend['count_date'].dt.dayofweek
spend['day']         = spend['count_date'].dt.day_name()
spend['count_week']  = spend['count_date'].to_numpy().astype('datetime64[W]')
spend['count_month'] = spend['count_date'].to_numpy().astype('datetime64[M]')
spend['week']        = spend['count_date'].dt.isocalendar().week
spend['month']       = spend['count_date'].dt.month
spend['yr']          = spend['count_date'].dt.year

cpi_table.rename(columns = {'CPI_Index': 'cpi_index', 'Month': 'month'}, inplace = True)

spend = inflation_adjust(spend, cpi_table, 2018) # adjust for inflation with 2018 as baseline
spend = mcard_adjust(spend) # adjust for cash to card shift

In [ ]:
# create caz and bid spend dataframes
caz_spend = spend[(spend['CAZ'] == 'CAZ') &
                  (spend['hours'].isin(hours)) &
                  (spend['week'] >= 10) &
                  (spend['week'] <= 22)]
caz_spend['month'] = caz_spend['month'].apply(lambda x: 5 if x == 6 else x)

bid_spend = spend[~(spend['bid_name'].isnull()) &
                  (spend['hours'].isin(hours)) &
                  (spend['week'] >= 10) &
                  (spend['week'] <= 22)]
bid_spend['month'] = bid_spend['month'].apply(lambda x: 5 if x == 6 else x)

In [ ]:
# For summary, calculate november spend friday effect
nov_spend = spend[(spend['CAZ'] == 'CAZ') & (spend['count_month'] == '2023-11-01') & (spend['hours'].isin(hours))]\
                 .groupby(['yr', 'daynumber', 'day'])\
                 .sum()[['txn_amt', 'txn_cnt']]\
                 .reset_index()\
                 .rename(columns = {'txn_amt': 'Spend Amount', 'txn_cnt': 'Spend Volume'})

nov_spend = spend_indexer(nov_spend)
# nov_spend.head()

# **Change in the Central Activity Zone (CAZ)**

## **Net Friday Change**

In [ ]:
# calculate year on year spend of daily friday spend and transactions
# yoy_daiyly = daily_pp_23[['yr', 'week', 'day', 'Spend', 'Transaction Counts']].merge(daily_pp_24[['yr', 'week', 'day', 'Spend', 'Transaction Counts']], on = ['week', 'day'], how = 'left')
# yoy_daily['yoy_spend'] = np.round((yoy_daily['Spend_y']/yoy_daily['Spend_x'] - 1) * 100, 1)
# yoy_daily['yoy_tx'] = np.round((yoy_daily['Transaction Counts_y']/yoy_daily['Transaction Counts_x'] - 1) * 100, 1)
# yoy_dail

In [ ]:
# weekday spend change computation for Adrian
yoy_mon_spend = spend[(spend['CAZ'] == 'CAZ') & (spend['hours'].isin(hours)) & (spend['day'] == 'Monday')]\
                     .groupby(['count_month', 'count_date']).sum()[['txn_amt', 'txn_cnt']]\
                     .reset_index()\
                     .groupby('count_month').mean(numeric_only = True)[['txn_amt', 'txn_cnt']]\
                     .reset_index()\
                     .sort_values(by = 'count_month')

yoy_tue_spend = spend[(spend['CAZ'] == 'CAZ') & (spend['hours'].isin(hours)) & (spend['day'] == 'Tuesday')]\
                     .groupby(['count_month', 'count_date']).sum()[['txn_amt', 'txn_cnt']]\
                     .reset_index()\
                     .groupby('count_month').mean(numeric_only = True)[['txn_amt', 'txn_cnt']]\
                     .reset_index()\
                     .sort_values(by = 'count_month')

yoy_wed_spend = spend[(spend['CAZ'] == 'CAZ') & (spend['hours'].isin(hours)) & (spend['day'] == 'Wednesday')]\
                     .groupby(['count_month', 'count_date']).sum()[['txn_amt', 'txn_cnt']]\
                     .reset_index()\
                     .groupby('count_month').mean(numeric_only = True)[['txn_amt', 'txn_cnt']]\
                     .reset_index()\
                     .sort_values(by = 'count_month')

yoy_thu_spend = spend[(spend['CAZ'] == 'CAZ') & (spend['hours'].isin(hours)) & (spend['day'] == 'Thursday')]\
                     .groupby(['count_month', 'count_date']).sum()[['txn_amt', 'txn_cnt']]\
                     .reset_index()\
                     .groupby('count_month').mean(numeric_only = True)[['txn_amt', 'txn_cnt']]\
                     .reset_index()\
                     .sort_values(by = 'count_month')

yoy_fri_spend = spend[(spend['CAZ'] == 'CAZ') & (spend['hours'].isin(hours)) & (spend['day'] == 'Friday')]\
                     .groupby(['count_month', 'count_date']).sum()[['txn_amt', 'txn_cnt']]\
                     .reset_index()\
                     .groupby('count_month').mean(numeric_only = True)[['txn_amt', 'txn_cnt']]\
                     .reset_index()\
                     .sort_values(by = 'count_month')

yoy_wd_spend = spend[(spend['CAZ'] == 'CAZ') & (spend['hours'].isin(hours)) & ~(spend['day'].isin(['Saturday', 'Sunday']))]\
                     .groupby(['count_month', 'count_date']).sum()[['txn_amt', 'txn_cnt']]\
                     .reset_index()\
                     .groupby('count_month').mean(numeric_only = True)[['txn_amt', 'txn_cnt']]\
                     .reset_index()\
                     .sort_values(by = 'count_month')

for df in [yoy_mon_spend, yoy_tue_spend, yoy_wed_spend, yoy_thu_spend, yoy_fri_spend, yoy_wd_spend]:
    df['prev_yr_txn_amt'] = df['txn_amt'].shift(12)
    df['prev_yr_txn_cnt'] = df['txn_cnt'].shift(12)
    df['yoy_spend'] = np.round((df['txn_amt']/df['prev_yr_txn_amt'] - 1) * 100, 1)
    df['yoy_txn_cnt'] = np.round((df['txn_cnt']/df['prev_yr_txn_cnt'] - 1) * 100, 1)

# yoy_mon_spend.tail(3)

In [ ]:
daily_spend = spend[(spend['CAZ'] == 'CAZ') & (spend['hours'].isin(hours)) & (spend['count_month'] >= '2023-01-01')]\
                   .groupby(['count_date', 'count_month', 'count_week', 'week', 'day', 'yr'])\
                   .sum()[['txn_amt', 'txn_cnt']]\
                   .reset_index()\
                   .rename(columns = {'txn_amt': 'Spend', 'txn_cnt': 'Transaction Counts'})

daily_pp_23 = daily_spend[(daily_spend['day'] == 'Friday') & (daily_spend['yr'] == 2023) & (daily_spend['week'] <= 22)]
daily_pp_24 = daily_spend[(daily_spend['day'] == 'Friday') & (daily_spend['yr'] == 2024) & (daily_spend['week'] <= 22)]

fig = make_subplots(rows = 1, cols = 2)

fig.add_trace(go.Scatter(x = daily_pp_23['week'], y = daily_pp_23['Spend'], marker = dict(color = weekdays[2]), name = 'Fridays 2023'),
                         row = 1, col = 1)
fig.add_trace(go.Scatter(x = daily_pp_24['week'], y = daily_pp_24['Spend'], marker = dict(color = weekdays[4]), name = 'Fridays 2024'),
                         row = 1, col = 1)
fig.add_trace(go.Scatter(x = daily_pp_23['week'], y = daily_pp_23['Transaction Counts'], marker = dict(color = weekdays[2]), name = 2023, showlegend = False),
                         row = 1, col = 2)
fig.add_trace(go.Scatter(x = daily_pp_24['week'], y = daily_pp_24['Transaction Counts'], marker = dict(color = weekdays[4]), name = 2024, showlegend = False),
                         row = 1, col = 2)
fig.add_shape(type = 'rect', xref = "x", yref = 'paper', x0 = 10, x1 = 22, y0 = min(daily_spend[daily_spend['day'] == 'Friday']['Spend']), y1 = max(daily_spend[daily_spend['day'] == 'Friday']['Spend']), line = dict(color = 'rgba(0, 0, 0, 0)', width = 3),
              fillcolor = 'rgb(229,236,246)', layer = 'below', col = 1, row = 1)
fig.add_shape(type = 'rect', xref = "x", yref = 'paper', x0 = 10, x1 = 22, y0 = min(daily_spend[daily_spend['day'] == 'Friday']['Transaction Counts']), y1 = max(daily_spend[daily_spend['day'] == 'Friday']['Transaction Counts']), line = dict(color = 'rgba(0, 0, 0, 0)', width = 3),
              fillcolor = 'rgb(229,236,246)', layer = 'below', col = 2, row = 1)
annotations=[dict(xref = 'paper',  yref = 'paper', x = 1.09, y = -0.15, showarrow = False,
                  text = '**Spend Index Aggregated & Anonymised by Mastercard <br>Inflation & Cash-Card Shift Adjusted by the GLA        ', font = dict(size = 12)),
             dict(xref = 'paper',  yref = 'paper', x = 0.15, y = 1.05, showarrow = False, text = 'Spend Index', font = dict(size = 14.5)),
             dict(xref = 'paper',  yref = 'paper', x = 0.85, y = 1.05, showarrow = False, text = 'Transaction Counts Index', font = dict(size = 14.5))]
fig.update_layout(title_text = 'Friday Spend in the CAZ During the Pilot Period', template = 'plotly_white+draft',
                  annotations = annotations, width = 2000, height = 600)
fig.update_xaxes(showgrid = False, tickvals = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22],
                 ticktext = ['', '', 'January', '', '', '', 'February', '', '', '', 'March', '', '', '', 'April', '', '', '', 'May', '',  '', ''])
fig.update_yaxes(showgrid = False, showticklabels = False)
fig.show(renderer)

In [ ]:
# plot daily spend across London from pilot period 2023 to date
month_spend = caz_spend[(caz_spend['day'] == 'Friday')]\
                        .groupby(['count_date', 'count_month', 'month', 'yr'])\
                        .sum()[['txn_amt', 'txn_cnt']].reset_index()
month_spend['month'] = month_spend['month'].apply(lambda x: 5 if x == 6 else x)
month_spend = month_spend.groupby(['month', 'yr'])\
                         .mean(numeric_only = True)[['txn_amt', 'txn_cnt']]\
                         .reset_index()\
                         .rename(columns = {'txn_amt': 'Spend', 'txn_cnt': 'Transaction Counts'})

fig = make_subplots(rows = 1, cols = 2)

for i, year in enumerate([2023, 2024]):
    fig.add_trace(go.Bar(x = month_spend[month_spend['yr'] == year]['month'],
                         y = month_spend[month_spend['yr'] == year]['Spend'],
                         marker = dict(color = [weekdays[0], weekdays[4]][i]), name = year), row = 1, col = 1)
    fig.add_trace(go.Bar(x = month_spend[month_spend['yr'] == year]['month'],
                         y = month_spend[month_spend['yr'] == year]['Transaction Counts'],
                         marker = dict(color = [weekdays[0], weekdays[4]][i]), name = year, showlegend = False), row = 1, col = 2)

fig.add_shape(type = 'rect', xref = "x", yref = 'paper', x0 = '2.5', x1 = '5.5', y0 = 0, y1 = 550000, line = dict(color = 'rgba(0, 0, 0, 0)', width = 3),
              fillcolor = 'rgb(229,236,246)', layer = 'below', col = 1, row = 1)
fig.add_shape(type = 'rect', xref = "x", yref = 'paper', x0 = '2.5', x1 = '5.5', y0 = 0, y1 = 1200000, line = dict(color = 'rgba(0, 0, 0, 0)', width = 3),
              fillcolor = 'rgb(229,236,246)', layer = 'below', col = 2, row = 1)
annotations = [dict(xref = 'paper',  yref = 'paper', x = 1.06, y = -0.15, showarrow = False,
                    text = '**Spend Index Aggregated & Anonymised by Mastercard <br>Inflation & Cash-Card Shift Adjusted by the GLA        ', font = dict(size = 12)),
               dict(xref = 'paper',  yref = 'paper', x = 0.15, y = 1.05, showarrow = False, text = 'Spend Index', font = dict(size = 14.5)),
               dict(xref = 'paper',  yref = 'paper', x = 0.85, y = 1.05, showarrow = False, text = 'Transaction Counts Index', font = dict(size = 14.5))]
fig.update_layout(title_text = 'Year-on-Year Friday Spend in the CAZ During the Pilot Period', width = 2000, height = 600, annotations = annotations, template = 'plotly_white+draft')
fig.update_xaxes(tickvals = [3, 4, 5], ticktext = ['March', 'April', 'May'])
fig.update_yaxes(showgrid = False, showticklabels = False)
fig.show(renderer)

In [ ]:
# create and index '23 & '24 spend
pp23_spend = caz_spend[(caz_spend['yr'] == 2023) &
                       ~(caz_spend['daynumber'].isin([5, 6]))]\
                      .groupby(['yr', 'daynumber', 'day'])\
                      .sum()[['txn_amt', 'txn_cnt']]\
                      .reset_index()\
                      .sort_values(by = ['yr', 'daynumber'])\
                      .rename(columns = {'txn_amt': 'Spend Amount', 'txn_cnt': 'Spend Volume'})

pp24_spend = caz_spend[(caz_spend['yr'] == 2024) &
                       ~(caz_spend['daynumber'].isin([5, 6]))]\
                      .groupby(['yr', 'daynumber', 'day'])\
                      .sum()[['txn_amt', 'txn_cnt']]\
                      .reset_index()\
                      .sort_values(by = ['yr', 'daynumber'])\
                      .rename(columns = {'txn_amt': 'Spend Amount', 'txn_cnt': 'Spend Volume'})

# index fridays to Wednesdays
pp23_spend = spend_indexer(pp23_spend)
pp24_spend = spend_indexer(pp24_spend)
pp_spend = pd.concat([pp23_spend, pp24_spend])

In [ ]:
# plot pilot period weekday spend indexed to Wednesdays in 2023 v 2024
fig = make_subplots(rows = 1, cols = 2)

for i, day in enumerate(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']):
    fig.add_trace(go.Bar(x = pp_spend[pp_spend['day'] == day]['yr'],
                         y = pp_spend[pp_spend['day'] == day]['Spend Amount'],
                         marker = dict(color = weekdays_fri[i]), name = day, showlegend = False),
                  row = 1, col = 1)
    fig.add_trace(go.Bar(x = pp_spend[pp_spend['day'] == day]['yr'],
                         y = pp_spend[pp_spend['day'] == day]['Spend Volume'],
                         marker = dict(color = weekdays_fri[i]), name = day),
                  row = 1, col = 2)

fri_23_spend = pp_spend[(pp_spend['day'] == 'Friday') & (pp_spend['yr']== 2023)]['Spend Amount'].values[0]
fri_23_txn = pp_spend[(pp_spend['day'] == 'Friday') & (pp_spend['yr']== 2023)]['Spend Volume'].values[0]

fig.add_trace(go.Scatter(x = ['2022.5', '2024.5'], y = [fri_23_spend, fri_23_spend], showlegend = False, marker = dict(color = 'rgba(0, 0, 0, 0.3)', size = 0.1)),
              row = 1, col = 1)
fig.add_trace(go.Scatter(x = ['2022.5', '2024.5'], y = [fri_23_txn, fri_23_txn], marker = dict(color = 'rgba(0, 0, 0, 0.3)', size = 0.1), name = "Friday '23 Benchmark"),
              row = 1, col = 2)
annotations = [dict(xref = 'paper',  yref = 'paper', x = 1.12, y = -0.15, showarrow = False,
                    text = '**Spend Index Aggregated & Anonymised by Mastercard <br>Inflation & Cash-Card Shift Adjusted by the GLA        ', font = dict(size = 12)),
               dict(xref = 'paper',  yref = 'paper', x = 0.19, y = 1.05, showarrow = False, text = 'Spend Index', font = dict(size = 14.5)),
               dict(xref = 'paper',  yref = 'paper', x = 0.85, y = 1.05, showarrow = False, text = 'Transaction Counts Index', font = dict(size = 14.5)),
               dict(xref = 'paper',  yref = 'paper', x = 0.428, y = 0.9, showarrow = False, text = 'down 1.6%', font = dict(size = 12.5)),
               dict(xref = 'paper',  yref = 'paper', x = 0.993, y = 0.82, showarrow = False, text = 'up 2.6%', font = dict(size = 12.5))]

fig.update_layout(title = 'Weekday Daytime Spend in the CAZ during Off-Peak Trial', width = 2000, height = 600, annotations = annotations, template = 'plotly+draft')
fig.update_xaxes(nticks = 4)
fig.update_yaxes(showgrid = False, showticklabels = False)
fig.show(renderer)

## **Friday Change Relative to the Midweek**

In [ ]:
# plot pilot period weekday spend indexed to Wednesdays in 2023 v 2024
fig = make_subplots(rows = 1, cols = 2)

for i, day in enumerate(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']):
    fig.add_trace(go.Bar(x = pp_spend[pp_spend['day'] == day]['yr'],
                         y = pp_spend[pp_spend['day'] == day]['Spend Amount idx'],
                         marker = dict(color = weekdays[i]), name = day, showlegend = False),
                  row = 1, col = 1)
    fig.add_trace(go.Bar(x = pp_spend[pp_spend['day'] == day]['yr'],
                         y = pp_spend[pp_spend['day'] == day]['Spend Volume idx'],
                         marker = dict(color = weekdays[i]), name = day),
                  row = 1, col = 2)

fig.add_trace(go.Scatter(x = ['2022.5', '2024.5'], y = [100, 100], showlegend = False, marker = dict(color = 'rgba(0, 0, 0, 0.3)', size = 0.3)),
              row = 1, col = 1)
fig.add_trace(go.Scatter(x = ['2022.5', '2024.5'], y = [100, 100], marker = dict(color = 'rgba(0, 0, 0, 0.3)', size = 0.3), name = 'Wed (Index = 100)', ),
              row = 1, col = 2)
annotations=[dict(xref = 'paper', yref = 'paper', x = 1.11, y = -0.15, showarrow = False,
                  text = '**Spend Index Aggregated & Anonymised by Mastercard <br>Inflation & Cash-Card Shift Adjusted by the GLA        ', font = dict(size = 12)),
             dict(xref = 'paper', yref = 'paper', x = 0.15, y = 1.05, showarrow = False, text = 'Spend Index', font = dict(size = 14.5)),
             dict(xref = 'paper', yref = 'paper', x = 0.85, y = 1.05, showarrow = False, text = 'Transaction Counts Index', font = dict(size = 14.5)),
             dict(xref = 'paper', yref = 'paper', x = 0.21, y = 0.92, showarrow = False, text = '10.2% more      <br>than Wednesday', font = dict(size = 12)),
             dict(xref = 'paper', yref = 'paper', x = 0.44, y = 0.92, showarrow = False, text = '9.1% more       <br>than Wednesday', font = dict(size = 12)),
             dict(xref = 'paper', yref = 'paper', x = 0.79, y = 0.825, showarrow = False,  text = '12.2% less       <br> than Wednesday', font = dict(size = 12)),
             dict(xref = 'paper', yref = 'paper', x = 0.99, y = 0.82, showarrow = False,  text = '13.0% less        <br>than Wednesday', font = dict(size = 12))]

fig.update_layout(title = 'Weekday Daytime Spend in the CAZ during Pilot Period (Relative to Wednesdays, Weds = 100)', width = 2000, height = 600, annotations = annotations, template = 'plotly+draft')
fig.update_xaxes(nticks = 4)
fig.update_yaxes(showgrid = False, showticklabels = True, range = [0, 120])
fig.show(renderer)

# **Cummulative Change in the BIDs**

## **Net Friday Spend**

In [ ]:
# compute retail spend in the BIDs on each weekday during the pilot period
# In 2023
pp23_bid_spend = bid_spend[(bid_spend['yr'] == 2023) &
                           ~(bid_spend['daynumber'].isin([5, 6]))]\
                          .groupby(['bid_name', 'day', 'daynumber'])\
                          .sum()[['txn_amt', 'txn_cnt']]\
                          .reset_index()\
                          .sort_values(by = ['bid_name', 'daynumber'])\
                          .rename(columns = {'txn_amt': 'Spend Amount', 'txn_cnt': 'Spend Volume'})

# In 2024
pp24_bid_spend = bid_spend[(bid_spend['yr'] == 2024) &
                           ~(bid_spend['daynumber'].isin([5, 6]))]\
                          .groupby(['bid_name', 'day', 'daynumber'])\
                          .sum()[['txn_amt', 'txn_cnt']]\
                          .reset_index()\
                          .sort_values(by = ['bid_name', 'daynumber'])\
                          .rename(columns = {'txn_amt': 'Spend Amount', 'txn_cnt': 'Spend Volume'})

# join '23 with '24 for YoY computation
pp_bid_spend = pp23_bid_spend[pp23_bid_spend['daynumber'].isin([2, 4])].merge(pp24_bid_spend[pp24_bid_spend['daynumber'].isin([2, 4])],
                                                                              on = ['bid_name', 'day', 'daynumber'], how = 'left', suffixes = ['_23', '_24'])

# pivot CAZ and BID dataframes
spend_fe_bid = pd.pivot_table(pp_bid_spend, index = ['bid_name'], columns = 'day', aggfunc = 'first',
                              values = ['Spend Amount_23', 'Spend Amount_24', 'Spend Volume_23', 'Spend Volume_24'])

# Compute % Friday activity relative to the midweek in pilot period 2023 & 2024 for the BIDs
spend_fe_bid['% Wed Net Spend Amount'] = ((spend_fe_bid[('Spend Amount_24', 'Wednesday')]/spend_fe_bid[('Spend Amount_23', 'Wednesday')]) - 1) * 100
spend_fe_bid['% Wed Net Spend Volume'] = ((spend_fe_bid[('Spend Volume_24', 'Wednesday')]/spend_fe_bid[('Spend Volume_23', 'Wednesday')]) - 1) * 100
spend_fe_bid['% Fri Net Spend Amount'] = ((spend_fe_bid[('Spend Amount_24', 'Friday')]/spend_fe_bid[('Spend Amount_23', 'Friday')]) - 1) * 100
spend_fe_bid['% Fri Net Spend Volume'] = ((spend_fe_bid[('Spend Volume_24', 'Friday')]/spend_fe_bid[('Spend Volume_23', 'Friday')]) - 1) * 100
spend_fe_bid['FE_Spend Amount_23'] = ((spend_fe_bid[('Spend Amount_23', 'Friday')]/spend_fe_bid[('Spend Amount_23', 'Wednesday')]) - 1) * 100
spend_fe_bid['FE_Spend Amount_24'] = ((spend_fe_bid[('Spend Amount_24', 'Friday')]/spend_fe_bid[('Spend Amount_24', 'Wednesday')]) - 1) * 100
spend_fe_bid['FE_Spend Volume_23'] = ((spend_fe_bid[('Spend Volume_23', 'Friday')]/spend_fe_bid[('Spend Volume_23', 'Wednesday')]) - 1) * 100
spend_fe_bid['FE_Spend Volume_24'] = ((spend_fe_bid[('Spend Volume_24', 'Friday')]/spend_fe_bid[('Spend Volume_24', 'Wednesday')]) - 1) * 100

In [ ]:
# Drop spend index values
spend_fe_bid = spend_fe_bid.droplevel(1, axis = 1).reset_index()\
                           [['bid_name', '% Wed Net Spend Amount', '% Wed Net Spend Volume', '% Fri Net Spend Amount', '% Fri Net Spend Volume', 'FE_Spend Amount_23', 'FE_Spend Amount_24', 'FE_Spend Volume_23', 'FE_Spend Volume_24']]\
                           .sort_values(by = 'bid_name')

# Calculate pt change in % Friday activity relative to the midweek across the BIDs
spend_fe_bid['FE_Change_Spend Amount'] = spend_fe_bid['FE_Spend Amount_24'] - spend_fe_bid['FE_Spend Amount_23']
spend_fe_bid['FE_Change_Spend Volume'] = spend_fe_bid['FE_Spend Volume_24'] - spend_fe_bid['FE_Spend Volume_23']

In [ ]:
bid_spatial_plot(spend_fe_bid, '% Fri Net Spend Amount', title = 'Change in Friday Spend Across the BIDs', vmin = -20, vmax = 20)
tc_bids = bids[bids['Type'] == 'Town Centre Bids']['Lookup BID Name'].values
caz_bids = bids[bids['Area'] == 'CAZ']['Lookup BID Name'].values
fig = plt.figure(figsize = (8, 6))
spend_fe_bid['% Fri Net Spend Amount'].hist(color = '#D7D8D9', bins = 30)
spend_fe_bid[spend_fe_bid['bid_name'].isin(tc_bids)]['% Fri Net Spend Amount'].hist(color = '#6DA7DE', bins = 30)
spend_fe_bid[spend_fe_bid['bid_name'].isin(caz_bids)]['% Fri Net Spend Amount'].hist(color = '#9E0059', bins = 30)
plt.title('Net Year on Year Change in Friday daytime Spend Across the BIDs')
plt.legend(['London BIDs', 'Town Centre BIDs', 'CAZ BIDs'])
plt.annotate('**Spend Index Aggregated & Anonymised by Mastercard \n   Inflation & Cash-Card Shift Adjusted by the GLA    ', xy = (170, 0), fontsize = 8)
plt.annotate('DRAFT - CONFIDENTIAL', xy = (0, 25), fontsize = 20, color = (0, 0, 0, 0.05))
plt.box(False)
plt.grid(False)
plt.show()
print('{} BIDs across London saw increased Friday daytime Spend during the pilot period year on year (i.e. March - May 2023 vs same period 2024).'.format(C.b + str(spend_fe_bid[spend_fe_bid['% Fri Net Spend Amount'] > 0].shape[0]) + C.e))
print('{} of the {} BIDs with Daytime Friday spend increase were town centre BIDs.'.format(C.b + str(spend_fe_bid[(spend_fe_bid['% Fri Net Spend Amount'] > 0) & (spend_fe_bid['bid_name'].isin(tc_bids))].shape[0]) + C.e, spend_fe_bid[spend_fe_bid['% Fri Net Spend Amount'] > 0].shape[0]))
print('Top 5 town centre BIDs were:')
spend_fe_bid[(spend_fe_bid['% Fri Net Spend Amount'] > 0) & (spend_fe_bid['bid_name'].isin(tc_bids))]\
            .sort_values(by = '% Fri Net Spend Amount', ascending = False).reset_index()[['bid_name', '% Fri Net Spend Amount']]\
            .rename(columns = {'bid_name': 'BID Name', '% Fri Net Spend Amount': '% Change in Friday Spend'}).head()

In [ ]:
print('\n{} BIDs across London saw decreased Friday Spend during the pilot period year on year (i.e. March - May 2023 vs same period 2024).'.format(C.b + str(spend_fe_bid[spend_fe_bid['% Fri Net Spend Amount'] < 0].shape[0]) + C.e))
print('{} of the {} BIDs with Friday spend decrease were town centre BIDs.'.format(C.b + str(spend_fe_bid[(spend_fe_bid['% Fri Net Spend Amount'] < 0) & (spend_fe_bid['bid_name'].isin(tc_bids))].shape[0]) + C.e, spend_fe_bid[spend_fe_bid['% Fri Net Spend Amount'] < 0].shape[0]))
print('The 5 town centre BIDs with biggest Friday spend decline include:')
spend_fe_bid[(spend_fe_bid['% Fri Net Spend Amount'] < 0) & (spend_fe_bid['bid_name'].isin(tc_bids))]\
            .sort_values(by = '% Fri Net Spend Amount', ascending = True).reset_index()[['bid_name', '% Fri Net Spend Amount']]\
            .rename(columns = {'bid_name': 'BID Name', '% Fri Net Spend Amount': '% Change in Friday Spend'}).head()

In [ ]:
bid_spatial_plot(spend_fe_bid, '% Fri Net Spend Volume', title = 'Change in Friday Transaction Counts Across the BIDs', vmin = -20, vmax = 20)
fig = plt.figure(figsize = (8, 6))
spend_fe_bid['% Fri Net Spend Volume'].hist(color = '#D7D8D9', bins = 30)
spend_fe_bid[spend_fe_bid['bid_name'].isin(tc_bids)]['% Fri Net Spend Volume'].hist(color = '#6DA7DE', bins = 30)
spend_fe_bid[spend_fe_bid['bid_name'].isin(caz_bids)]['% Fri Net Spend Volume'].hist(color = '#9E0059', bins = 30)
plt.title('Daytime Friday Transaction Counts Change Across the BIDs')
plt.legend(['London BIDs', 'Town Centre BIDs', 'CAZ BIDs'])
plt.box(False)
plt.annotate('**Spend Index Aggregated & Anonymised by Mastercard \n   Inflation & Cash-Card Shift Adjusted by the GLA     ', xy = (100, 0), fontsize = 8)
plt.annotate('DRAFT - CONFIDENTIAL', xy = (-25, 15), fontsize = 20, color = (0, 0, 0, 0.05))
plt.grid(False)
plt.show()
print('{} BIDs across London saw more Friday transactions during the pilot period year on year (i.e. March - May 2023 vs same period 2024).'.format(C.b + str(spend_fe_bid[spend_fe_bid['% Fri Net Spend Volume'] > 0].shape[0]) + C.e))
print('{} of those {} BIDs with Friday transactions increase were town centre BIDs.'.format(C.b + str(spend_fe_bid[(spend_fe_bid['% Fri Net Spend Volume'] > 0) & (spend_fe_bid['bid_name'].isin(tc_bids))].shape[0]) + C.e, spend_fe_bid[spend_fe_bid['% Fri Net Spend Volume'] > 0].shape[0]))
print('Top 5 town centre BIDs were:')
spend_fe_bid[(spend_fe_bid['% Fri Net Spend Volume'] > 0) & (spend_fe_bid['bid_name'].isin(tc_bids))]\
            .sort_values(by = '% Fri Net Spend Volume', ascending = False).reset_index()[['bid_name', '% Fri Net Spend Volume']]\
            .rename(columns = {'bid_name': 'BID Name', '% Fri Net Spend Volume': '% Change in Friday Transaction Counts'}).head()

In [ ]:
print('\n{} BIDs across London saw less Friday transactions during the pilot period year on year (i.e. March - May 2023 vs same period 2024).'.format(C.b + str(spend_fe_bid[spend_fe_bid['% Fri Net Spend Volume'] < 0].shape[0]) + C.e))
print('{} of the {} BIDs with Friday transactions decline were town centre BIDs.'.format(C.b + str(spend_fe_bid[(spend_fe_bid['% Fri Net Spend Volume'] < 0) & (spend_fe_bid['bid_name'].isin(tc_bids))].shape[0]) + C.e, spend_fe_bid[spend_fe_bid['% Fri Net Spend Volume'] < 0].shape[0]))
print('The 5 town centre BIDs with the highest transactions decline were:')
spend_fe_bid[(spend_fe_bid['% Fri Net Spend Volume'] < 0) & (spend_fe_bid['bid_name'].isin(tc_bids))]\
            .sort_values(by = '% Fri Net Spend Volume', ascending = True).reset_index()[['bid_name', '% Fri Net Spend Volume']]\
            .rename(columns = {'bid_name': 'BID Name', '% Fri Net Spend Volume': '% Change in Friday Transaction Counts'}).head()

## **Friday Change Relative to the Midweek**

In [ ]:
bid_spatial_plot(spend_fe_bid, 'FE_Change_Spend Amount', title = 'Change in Friday Spend Relative to the Midweek', vmin = -20, vmax = 20)

spend_fe_bid = spend_fe_bid.sort_values(by = 'FE_Spend Amount_24').reset_index().rename(columns = {'index': 'number'}).reset_index()
plt.figure(figsize = (15, 10))
plt.scatter(spend_fe_bid['FE_Spend Amount_24'], spend_fe_bid['index'], cmap = weekdays[2], label = '2024 Friday Proportion of Midweek Spend')
plt.scatter(spend_fe_bid['FE_Spend Amount_23'], spend_fe_bid['index'], cmap = weekdays[4], label = '2023 Friday Proportion of Midweek Spend')
plt.plot([0, 0], [0, spend_fe_bid['index'].max()], c = 'black', alpha = .7)
for j in range(spend_fe_bid.shape[0]):
    a, b, c, d = zip(spend_fe_bid[['FE_Spend Amount_24', 'index', 'FE_Spend Amount_23', 'index']].iloc[j, :])
    plt.plot([a, c], [b, d], c = 'black', alpha = .3)
plt.title('Year-on-Year Change in Trial Friday Spend Relative to the Midweek per BID', fontweight = "bold", fontsize = 16)
plt.annotate('**Spend Index Aggregated & Anonymised by Mastercard \n   Inflation & Cash-Card Shift Adjusted by the GLA     ', xy = (210, -2), fontsize = 8)
plt.annotate('DRAFT - CONFIDENTIAL', xy = (20, 40), fontsize = 50, color = (0, 0, 0, 0.05))
plt.legend(loc = 'center right', fontsize = 'large')
plt.show()

In [ ]:
print('1. Only {} BIDs had lower Friday spend compared to the midweek in the pilot period, {} of which saw declined Friday proportions from 2023 levels.'.format(
      len(spend_fe_bid[spend_fe_bid['FE_Spend Amount_24'] < 0]),
      len(spend_fe_bid[(spend_fe_bid['FE_Spend Amount_24'] < 0) & (spend_fe_bid['FE_Spend Amount_24'] < spend_fe_bid['FE_Spend Amount_23'])])
))
print('2. {} BIDs saw an improvement in Friday spend relative to the midweek during the off-peak trial, {} of which were town centre BIDs and {}, BIDs in the CAZ.'.format(
       len(spend_fe_bid[spend_fe_bid['FE_Change_Spend Amount'] > 0]),
       len(spend_fe_bid[(spend_fe_bid['FE_Change_Spend Amount'] > 0) & (spend_fe_bid['bid_name'].isin(tc_bids))]),
       len(spend_fe_bid[(spend_fe_bid['FE_Change_Spend Amount'] > 0) & (spend_fe_bid['bid_name'].isin(caz_bids))])
))
print('3. Friday spend maintained a higher spend or exceeded midweek spend during the pilot period in {} of the {} BIDs with increased Friday to midweek spend proportions.'.format(
      len(spend_fe_bid[(spend_fe_bid['FE_Change_Spend Amount'] > 0) & (spend_fe_bid['FE_Spend Amount_23'] > 0)]),
      len(spend_fe_bid[spend_fe_bid['FE_Change_Spend Amount'] > 0])
))

In [ ]:
bid_spatial_plot(spend_fe_bid, 'FE_Change_Spend Volume', title = 'Change in Friday Transaction Counts Relative to the Midweek', vmin = -20, vmax = 20)
spend_fe_bid = spend_fe_bid.sort_values(by = 'FE_Spend Volume_24').reset_index().rename(columns = {'index': 'number 1'}).reset_index()
plt.figure(figsize = (15, 10))
plt.scatter(spend_fe_bid['FE_Spend Volume_24'], spend_fe_bid['index'], cmap = weekdays[2], label = '2024 Friday Proportion of Midweek Transaction Counts')
plt.scatter(spend_fe_bid['FE_Spend Volume_23'], spend_fe_bid['index'], cmap = weekdays[4], label = '2023 Friday Proportion of Midweek Transaction Counts')
plt.plot([0, 0], [0, spend_fe_bid['index'].max()], c = 'black', alpha = .7)
for j in range(spend_fe_bid.shape[0]):
    a, b, c, d = zip(spend_fe_bid[['FE_Spend Volume_24', 'index', 'FE_Spend Volume_23', 'index']].iloc[j, :])
    plt.plot([a, c], [b, d], c = 'black', alpha = .3)
plt.title('Year-on-Year Change in Trial Friday Transaction Counts Relative to the Midweek per BID', fontweight = "bold", fontsize = 16)
plt.annotate('**Spend Index Aggregated & Anonymised by Mastercard \n   Inflation & Cash-Card Shift Adjusted by the GLA     ', xy = (55, -2), fontsize = 8)
plt.annotate('DRAFT - CONFIDENTIAL', xy = (-20, 40), fontsize = 50, color = (0, 0, 0, 0.05))
plt.legend(loc = 'center right', fontsize = 'large')
plt.show()

In [ ]:
print('1. {} BIDs had fewer Friday transactions compared to the midweek in the pilot period, {} of which saw declined Friday proportions from 2023 levels.'.format(
      len(spend_fe_bid[spend_fe_bid['FE_Spend Volume_24'] < 0]),
      len(spend_fe_bid[(spend_fe_bid['FE_Spend Volume_24'] < 0) & (spend_fe_bid['FE_Spend Volume_24'] < spend_fe_bid['FE_Spend Volume_23'])])
))
print('2. {} BIDs saw an improvement in Friday transactions relative to the midweek during the off-peak trial, {} of which were town centre BIDs and {}, BIDs in the CAZ.'.format(
       len(spend_fe_bid[spend_fe_bid['FE_Change_Spend Volume'] > 0]),
       len(spend_fe_bid[(spend_fe_bid['FE_Change_Spend Volume'] > 0) & (spend_fe_bid['bid_name'].isin(tc_bids))]),
       len(spend_fe_bid[(spend_fe_bid['FE_Change_Spend Volume'] > 0) & (spend_fe_bid['bid_name'].isin(caz_bids))])
))
print('3. Friday spend either maintained more transactions or exceeded midweek transaction counts during the pilot period in {} of these {} BIDs.'.format(
      len(spend_fe_bid[(spend_fe_bid['FE_Change_Spend Volume'] > 0) & (spend_fe_bid['FE_Spend Volume_23'] > 0)]),
      len(spend_fe_bid[spend_fe_bid['FE_Change_Spend Volume'] > 0])
))

# **Appendix (Change in HSDS BIDs)**

In [ ]:
def increase_or_decrease(df):
    for col in ['% Wed Net Spend Amount', '% Wed Net Spend Volume', '% Fri Net Spend Amount', '% Fri Net Spend Volume']:
        df[col + ' iord'] = df[col].apply(lambda x: 'increased' if x > 0 else 'decreased' if x < 0 else 'stayed the same')
    df['FE_Change_Spend Amount' + ' iord'] = df.apply(lambda x: 'increased {}pts'.format(np.round(x[6] - x[5], 1)) if x[6] > x[5] else
                                                                'decreased {}pts'.format(np.round(x[5] - x[6], 1)) if x[5] > x[6] else
                                                                'stayed the same', axis = 1)
    df['FE_Change_Spend Amount' + ' iord comment'] = df.apply(lambda x: 'There was no friday effect' if (x[5] > 0) & (x[6] > 0) else
                                                                        'Fridays overtook the midweek' if (x[5] < 0) & (x[6] > 0) else
                                                                        'Friday Effect remained', axis = 1)
    df['FE_Change_Spend Volume' + ' iord'] = df.apply(lambda x: 'increased {}pts'.format(np.round(x[8] - x[7], 1)) if x[8] > x[7] else
                                                                'decreased {}pts'.format(np.round(x[7] - x[8], 1)) if x[7] > x[8] else
                                                                'stayed the same', axis = 1)
    df['FE_Change_Spend Volume' + ' iord comment'] = df.apply(lambda x: 'There was no friday effect' if (x[7] > 0) & (x[8] > 0) else
                                                                        'Fridays overtook the midweek' if (x[7] < 0) & (x[8] > 0) else
                                                                        'Friday Effect remained', axis = 1)
increase_or_decrease(spend_fe_bid)

In [ ]:
# create single bid spend metric dataset
pp23_bid_spend['Year'] = 2023
pp24_bid_spend['Year'] = 2024
pp_bid_spend = pp_bid_spend[pp_bid_spend['bid_name'].isin(hsds_bids)]
pp_bid_spend = pd.concat([pp23_bid_spend, pp24_bid_spend])

# calculate daily spend in the BIDs

daily_bid_spend = spend[(spend['bid_name'].isin(hsds_bids)) &
                        (spend['day'] == 'Friday') &
                        (spend['hours'].isin(hours)) &
                        (spend['week'] <= 22)]\
                       .groupby(['bid_name', 'yr', 'week']).sum()[['txn_amt', 'txn_cnt']].reset_index()
daily_bid_spend.rename(columns = {'yr': 'Year', 'txn_amt': 'Spend', 'txn_cnt': 'Transaction Counts'}, inplace = True)
# daily_bid_spend

In [ ]:
# create BID level plots and summaries of changes in net Friday spend amount, net Friday spend volume, Friday spend amount indexed to wednesday and Friday spend volume indexed to wednesday

for bid in hsds_bids:
    # print bid names as headers and each change category as subheaders
    printmd(f'## {bid}')

    # plot yoy spend and transactions counts during the pilot period
    fig = make_subplots(rows = 1, cols = 2)
    for i, year in enumerate([2023, 2024]):
        fig.add_trace(go.Scatter(x = daily_bid_spend[(daily_bid_spend['bid_name'] == bid) & (daily_bid_spend['Year'] == year)]['week'],
                                 y = daily_bid_spend[(daily_bid_spend['bid_name'] == bid) & (daily_bid_spend['Year'] == year)]['Spend'],
                                 marker = dict(color = [weekdays[2], weekdays[4]][i]), name = 'Fridays ' + str(year), showlegend = False),
                      row = 1, col = 1)
        fig.add_trace(go.Scatter(x = daily_bid_spend[(daily_bid_spend['bid_name'] == bid) & (daily_bid_spend['Year'] == year)]['week'],
                                 y = daily_bid_spend[(daily_bid_spend['bid_name'] == bid) & (daily_bid_spend['Year'] == year)]['Transaction Counts'],
                                 marker = dict(color = [weekdays[2], weekdays[4]][i]), name = 'Fridays ' + str(year)),
                      row = 1, col = 2)
    fig.add_shape(type = 'rect', xref = "x", yref = 'paper', x0 = 10, x1 = 22,
                  y0 = min(daily_bid_spend[daily_bid_spend['bid_name'] == bid]['Spend']),
                  y1 = max(daily_bid_spend[daily_bid_spend['bid_name'] == bid]['Spend']),
                  line = dict(color = 'rgba(0, 0, 0, 0)', width = 3), fillcolor = 'rgb(229,236,246)', layer = 'below', col = 1, row = 1)
    fig.add_shape(type = 'rect', xref = "x", yref = 'paper', x0 = 10, x1 = 22,
                  y0 = min(daily_bid_spend[daily_bid_spend['bid_name'] == bid]['Transaction Counts']),
                  y1 = max(daily_bid_spend[daily_bid_spend['bid_name'] == bid]['Transaction Counts']),
                  line = dict(color = 'rgba(0, 0, 0, 0)', width = 3), fillcolor = 'rgb(229,236,246)', layer = 'below', col = 2, row = 1)
    annotations = [dict(xref = 'paper',  yref = 'paper', x = 1.05, y = -0.15, showarrow = False,
                      text = '**Spend Index Aggregated & Anonymised by Mastercard <br>Inflation & Cash-Card Shift Adjusted by the GLA       ', font = dict(size = 12)),
                   dict(xref = 'paper',  yref = 'paper', x = 0.20, y = 1.05, showarrow = False, text = 'Spend Index', font = dict(size = 14.5)),
                   dict(xref = 'paper',  yref = 'paper', x = 0.85, y = 1.05, showarrow = False, text = 'Transaction Counts Index', font = dict(size = 14.5))]
    fig.update_layout(title = 'Friday Daytime Spend in {} During the Pilot Period'.format(bid), annotations = annotations, height = 600, width = 2000, template = 'plotly_white+draft')
    fig.update_xaxes(showgrid = False, tickvals = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22],
                    ticktext = ['', '', 'January', '', '', '', 'February', '', '', '', 'March', '', '', '', 'April', '', '', '', 'May', '',  '', ''])
    fig.update_yaxes(showgrid = False, nticks = 5)
    fig.show(renderer)

    # plot yoy spend and transaction count changes during the pilot period
    fig = make_subplots(rows = 1, cols = 2)
    for i, day in enumerate(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']):
        fig.add_trace(go.Bar(x = pp_bid_spend[(pp_bid_spend['bid_name'] == bid) & (pp_bid_spend['day'] == day)]['Year'],
                             y = pp_bid_spend[(pp_bid_spend['bid_name'] == bid) & (pp_bid_spend['day'] == day)]['Spend Amount'],
                             marker = dict(color = weekdays[i]), name = day, showlegend = False),
                      row = 1, col = 1)
        fig.add_trace(go.Bar(x = pp_bid_spend[(pp_bid_spend['bid_name'] == bid) & (pp_bid_spend['day'] == day)]['Year'],
                             y = pp_bid_spend[(pp_bid_spend['bid_name'] == bid) & (pp_bid_spend['day'] == day)]['Spend Volume'],
                             marker = dict(color = weekdays[i]), name = day),
                      row = 1, col = 2)

    fri_23_spend = pp_bid_spend[(pp_bid_spend['bid_name'] == bid) & (pp_bid_spend['day'] == 'Friday')]['Spend Amount'].values[0]
    fri_23_txn = pp_bid_spend[(pp_bid_spend['bid_name'] == bid) & (pp_bid_spend['day'] == 'Friday')]['Spend Volume'].values[0]

    fig.add_trace(go.Scatter(x = ['2022.5', '2024.5'], y = [fri_23_spend, fri_23_spend], showlegend = False, marker = dict(color = 'rgba(0, 0, 0, 0.3)', size = 0.1)),
                  row = 1, col = 1)
    fig.add_trace(go.Scatter(x = ['2022.5', '2024.5'], y = [fri_23_txn, fri_23_txn], marker = dict(color = 'rgba(0, 0, 0, 0.3)', size = 0.1), name = "Friday '23 Benchmark"),
                  row = 1, col = 2)
    annotations = [dict(xref = 'paper',  yref = 'paper', x = 1.08, y = -0.15, showarrow = False,
                      text = '**Spend Index Aggregated & Anonymised by Mastercard <br>Inflation & Cash-Card Shift Adjusted by the GLA       ', font = dict(size = 12)),
                   dict(xref = 'paper',  yref = 'paper', x = 0.20, y = 1.05, showarrow = False, text = 'Spend Index', font = dict(size = 14.5)),
                   dict(xref = 'paper',  yref = 'paper', x = 0.85, y = 1.05, showarrow = False, text = 'Transaction Counts Index', font = dict(size = 14.5))]
    fig.update_layout(title = 'Weekday Daytime Spend in {}'.format(bid), annotations = annotations, height = 600, width = 2000, template = 'plotly+draft')
    fig.update_xaxes(nticks = 4)
    fig.update_yaxes(showgrid = False, nticks = 5)
    fig.show(renderer)

    printmd(f'### **Changes in Spend**')

    # create variables
    spend_in_2023 = pp23_bid_spend[pp23_bid_spend['bid_name'] == bid]['Spend Amount'].sum()
    spend_in_2024 = pp24_bid_spend[pp24_bid_spend['bid_name'] == bid]['Spend Amount'].sum()
    pct_wd_spend_change = np.round(((spend_in_2024/spend_in_2023) - 1) * 100, 1)

    if (pct_wd_spend_change == 0.0) or (pct_wd_spend_change == -0.0):
        print('1. Overall spend in {} on weekdays between March to May 2024 {} marginally compared to the same period in 2023.'.format(bid, 'increased' if pct_wd_spend_change > 0 else 'decreased'))
    else:
        print('1. Overall spend in {} on weekdays between March to May 2024 {} {}% compared to the same period in 2023.'.format(bid, 'increased' if pct_wd_spend_change > 0 else 'decreased', pct_wd_spend_change))

    # create variables to return % change in spend amount and direction of change in spend amount
    fri_spend_trend = spend_fe_bid[spend_fe_bid['bid_name'] == bid]['% Fri Net Spend Amount iord'].values[0]
    fri_spend_change = spend_fe_bid[spend_fe_bid['bid_name'] == bid]['% Fri Net Spend Amount'].values[0]

    # print statement summarizing change in net friday spend
    print('2. Net spend on {} in the pilot period {} by {}% compared to Fridays in the same period in 2023.'.format(C.b+'Fridays'+C.e, fri_spend_trend, np.round(fri_spend_change, 1)))

    # create variables
    max_weekday_23_spend =   pp23_bid_spend[pp23_bid_spend['bid_name'] == bid]['Spend Amount'].max()                                                                 # max weekday spend in 2023
    max_weekday_24_spend =   pp24_bid_spend[pp24_bid_spend['bid_name'] == bid]['Spend Amount'].max()                                                                 # max weekday spend in 2024
    weekday_w_max_23_spend = pp23_bid_spend[(pp23_bid_spend['bid_name'] == bid) & (pp23_bid_spend['Spend Amount'] == max_weekday_23_spend)]['day'].values[0]    # weekday with max spend in 2023
    weekday_w_max_24_spend = pp24_bid_spend[(pp24_bid_spend['bid_name'] == bid) & (pp24_bid_spend['Spend Amount'] == max_weekday_24_spend)]['day'].values[0]    # weekday with max spend in 2024
    fri_23_spend =           pp23_bid_spend[(pp23_bid_spend['bid_name'] == bid) & (pp23_bid_spend['day'] == 'Friday')]['Spend Amount'].values[0]                          # friday spend in 2023
    fri_24_spend =           pp24_bid_spend[(pp24_bid_spend['bid_name'] == bid) & (pp24_bid_spend['day'] == 'Friday')]['Spend Amount'].values[0]                          # friday spend in 2024

    if (max_weekday_23_spend == fri_23_spend) & (max_weekday_24_spend == fri_24_spend):                                  # if friday is the weekday with the highest spend in both 2023 and 2024
        print('3. Spend in the BID remained at its highest on {} compared to every other weekday across both years.'.format(C.b+'Fridays'+C.e))
    elif (max_weekday_23_spend == fri_23_spend) & ~(max_weekday_24_spend == fri_24_spend):                            # if friday was the weekday with the highest spend in 2023 but not in 2024
        print('3. {} in the 2024 pilot period lost the lead in Spend to {}.'.format(C.b+'Fridays'+C.e, weekday_w_max_24_spend))
    elif ~(max_weekday_23_spend == fri_23_spend) & (max_weekday_24_spend == fri_24_spend):                        # if friday was not the weekday with the highest spend in 2023 but was in 2024
        print('3. {} took over the lead from {}s as weekday with the highest spend in the 2024 pilot period.'.format(C.b+'Fridays'+C.e, weekday_w_max_23_spend))
    elif ~(max_weekday_23_spend == fri_23_spend) & ~(max_weekday_24_spend == fri_24_spend):                              # if friday was not the weekday with the highest spend in 2023 nor 2024
        if weekday_w_max_23_spend == weekday_w_max_24_spend:                                                                         # if the same weekday had the highest spend in both periods
            print('3. {}s maintained the lead in Spend during both periods in this BID.'.format(weekday_w_max_23_spend))
        else:                                                                                                                           # if spend is highest during the midweek in both periods
            print('3. In this BID, spend was highest during the midweek across both periods.'.format(weekday_w_max_23_spend))

    # create variables
    wed_spend_trend = spend_fe_bid[spend_fe_bid['bid_name'] == bid]['% Wed Net Spend Amount iord'].values[0]                                                    # wednesday spend change in 2023
    wed_spend_change = spend_fe_bid[spend_fe_bid['bid_name'] == bid]['% Wed Net Spend Amount'].values[0]                                                        # wednesday spend change in 2024
    fe_spend_trend = spend_fe_bid[spend_fe_bid['bid_name'] == bid]['FE_Change_Spend Amount iord'].values[0]                                               # yoy direction of spend friday effect
    fe_spend_change = spend_fe_bid[spend_fe_bid['bid_name'] == bid]['FE_Change_Spend Amount'].values[0]                                                      # yoy change in spend friday effect

    # print statement summarizing change in net friday spend
    if (wed_spend_trend == fri_spend_trend) & (wed_spend_trend == fe_spend_trend):                  # if spend on wednesdays and fridays, and friday effect during the pilot period changed in a similar manner year on year
        print('4. Similar to Fridays, Wednesdays in the pilot period also {} year on year by {}%. \n5. However, Friday spend {} {} in the pilot period.'\
                  .format(wed_spend_trend, np.round(wed_spend_change, 1), C.b+'relative to the midweek'+C.e, fe_spend_trend))
    elif (wed_spend_trend == fri_spend_trend) & (wed_spend_trend != fe_spend_trend):                # if spend on wednesdays and fridays, and friday effect during the pilot period changed in a similar manner year on year
        print('4. Similar to Fridays, Wednesdays in the pilot period also {} year on year by {}%. \n5. As a result, Friday spend {} {} in the pilot period.'\
                  .format(wed_spend_trend, np.round(wed_spend_change, 1), C.b+'relative to the midweek'+C.e, fe_spend_trend))
    else:
        print('4. Unlike Fridays, Wednesdays in the pilot period {} year on year by {}%. \n5. As a result, Friday spend {} {} in the pilot period.'\
                  .format(wed_spend_trend, np.round(wed_spend_change, 1), C.b+'relative to the midweek'+C.e, fe_spend_trend))

    printmd(f'<br />')
    printmd(f'### **Changes in Transaction Counts**')

    # create variables
    txn_in_2023 = pp23_bid_spend[pp23_bid_spend['bid_name'] == bid]['Spend Volume'].sum()
    txn_in_2024 = pp24_bid_spend[pp24_bid_spend['bid_name'] == bid]['Spend Volume'].sum()
    pct_wd_txn_change = np.round(((txn_in_2024/txn_in_2023) - 1) * 100, 1)

    if (pct_wd_txn_change == -0.0) or (pct_wd_txn_change == 0.0):
        print('1. Total weekday transaction counts in {} between March to May 2024 {} marginally compared to the same period in 2023.'.format(bid, 'increased' if pct_wd_txn_change > 0 else 'decreased'))
    else:
        print('1. Total weekday transaction counts in {} between March to May 2024 {} {}% compared to the same period in 2023.'.format(bid, 'increased' if pct_wd_txn_change > 0 else 'decreased', pct_wd_txn_change))

    # create variables to return % change in txn counts and direction of change in txn counts
    fri_txn_trend = spend_fe_bid[spend_fe_bid['bid_name'] == bid]['% Fri Net Spend Volume iord'].values[0]
    fri_txn_change = spend_fe_bid[spend_fe_bid['bid_name'] == bid]['% Fri Net Spend Volume'].values[0]

    # print statement summarizing change in net friday txn counts
    print('2. Net transaction counts on {} in the pilot period {} by {}% compared to Fridays in the same period in 2023.'.format(C.b+'Fridays'+C.e, fri_txn_trend, np.round(fri_txn_change, 1)))

    # create variables
    max_weekday_23_txn =   pp23_bid_spend[pp23_bid_spend['bid_name'] == bid]['Spend Volume'].max()                                                                 # max weekday txn counts in 2023
    max_weekday_24_txn =   pp24_bid_spend[pp24_bid_spend['bid_name'] == bid]['Spend Volume'].max()                                                                 # max weekday txn counts in 2024
    weekday_w_max_23_txn = pp23_bid_spend[(pp23_bid_spend['bid_name'] == bid) & (pp23_bid_spend['Spend Volume'] == max_weekday_23_txn)]['day'].values[0]      # weekday with max txn counts in 2023
    weekday_w_max_24_txn = pp24_bid_spend[(pp24_bid_spend['bid_name'] == bid) & (pp24_bid_spend['Spend Volume'] == max_weekday_24_txn)]['day'].values[0]      # weekday with max txn counts in 2024
    fri_23_txn =           pp23_bid_spend[(pp23_bid_spend['bid_name'] == bid) & (pp23_bid_spend['day'] == 'Friday')]['Spend Volume'].values[0]                          # friday txn counts in 2023
    fri_24_txn =           pp24_bid_spend[(pp24_bid_spend['bid_name'] == bid) & (pp24_bid_spend['day'] == 'Friday')]['Spend Volume'].values[0]                          # friday txn counts in 2024

    if (max_weekday_23_txn == fri_23_txn) & (max_weekday_24_txn == fri_24_txn):                                        # if friday is the weekday with the highest txn counts in both 2023 and 2024
        print('3. Spend in the BID remained at its highest on {} compared to every other weekday across both years.'.format(C.b+'Fridays'+C.e))
    elif (max_weekday_23_txn == fri_23_txn) & ~(max_weekday_24_txn == fri_24_txn):                                  # if friday was the weekday with the highest txn counts in 2023 but not in 2024
        print('3. {} in the 2024 pilot period lost the lead in Spend to {}.'.format(C.b+'Fridays'+C.e, weekday_w_max_24_txn))
    elif ~(max_weekday_23_txn == fri_23_txn) & (max_weekday_24_txn == fri_24_txn):                              # if friday was not the weekday with the highest txn counts in 2023 but was in 2024
        print('3. {} took over the lead from {}s as weekday with the highest txn in the 2024 pilot period.'.format(C.b+'Fridays'+C.e, weekday_w_max_23_txn))
    elif ~(max_weekday_23_txn == fri_23_txn) & ~(max_weekday_24_txn == fri_24_txn):                                    # if friday was not the weekday with the highest txn counts in 2023 nor 2024
        if weekday_w_max_23_txn == weekday_w_max_24_txn:                                                                           # if the same weekday had the highest txn counts in both periods
            print('3. {}s maintained the lead in Spend during both periods in this BID.'.format(weekday_w_max_23_txn))
        else:                                                                                                                         # if txn counts is highest during the midweek in both periods
            print('3. In this BID, txn was highest during the midweek across both periods.'.format(weekday_w_max_23_txn))

    # create variables

    wed_txn_trend = spend_fe_bid[spend_fe_bid['bid_name'] == bid]['% Wed Net Spend Volume iord'].values[0]                                                       # wednesday txn counts change in 2023
    wed_txn_change = spend_fe_bid[spend_fe_bid['bid_name'] == bid]['% Wed Net Spend Volume'].values[0]                                                           # wednesday txn counts change in 2024
    fe_txn_trend = spend_fe_bid[spend_fe_bid['bid_name'] == bid]['FE_Change_Spend Volume iord'].values[0]                                                  # yoy direction of txn counts friday effect
    fe_txn_change = spend_fe_bid[spend_fe_bid['bid_name'] == bid]['FE_Change_Spend Volume'].values[0]                                                         # yoy change of txn counts friday effect

    # print statement summarizing change in net friday txn
    if (wed_txn_trend == fri_txn_trend) & (wed_txn_trend == fe_txn_trend):                     # if txn counts on wednesdays and fridays, and friday effect during the pilot period changed in a similar manner year on year
        print('4. Similar to Fridays, Wednesdays in the pilot period also {} year on year by {}%. \n5. However, Friday transactions {} {} in the pilot period.'\
                  .format(wed_txn_trend, np.round(wed_txn_change, 1), C.b+'relative to the midweek'+C.e, fe_txn_trend))
    elif (wed_txn_trend == fri_txn_trend) & (wed_txn_trend != fe_txn_trend):                 # if txn counts on wednesdays and fridays, and friday effect during the pilot period changed in a different manner year on year
        print('4. Similar to Fridays, Wednesdays in the pilot period also {} year on year by {}%. \n5. As a result, Friday transactions {} {} in the pilot period.'\
                  .format(wed_txn_trend, np.round(wed_txn_change, 1), C.b+'relative to the midweek'+C.e, fe_txn_trend))
    else:
        print('4. Unlike Fridays, Wednesdays in the pilot period {} year on year by {}%. \n5. Friday transactions {} {} in the pilot period.'\
                  .format(wed_txn_trend, np.round(wed_txn_change, 1), C.b+'relative to the midweek'+C.e, fe_txn_trend))

    # summary = input("Enter a brief overall summary on this BID's spend change patterns: ")
    # print('In summary, ', summary)

    printmd('_____________________________________________________')

In [ ]:
!jupyter nbconvert --to html --no-input "Reduced Fares Friday Pilot Review.ipynb"
# !jupyter nbconvert --to pdf --no-input "Reduced Fares Friday Pilot Review.ipynb"